# GRU Implementation of PCDCnet

In [ ]:
!pip install torch pandas numpy scikit-learn torch_geometric

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


## Data Pre-processing

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
import os # For constructing file paths

# --- Helper Function to Load and Preprocess Single Yearly Meteorological Files ---
def load_single_met_file(file_path, met_var_name, relevant_col_name, is_hourly):
    """
    Loads a single yearly meteorological CSV, standardizes columns,
    and performs daily aggregation if the data is hourly.
    """
    try:
        # Use low_memory=False to handle potentially mixed types in large CSVs
        df = pd.read_csv(file_path, low_memory=False)
    except FileNotFoundError:
        print(f"Warning: File not found {file_path}. Skipping this file.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Warning: File is empty {file_path}. Skipping this file.")
        return None


    # Standardize column names (assuming yearly files might use different casing)
    # Create a mapping based on common variations
    column_rename_map = {}
    # Date column
    if 'DATE' in df.columns: column_rename_map['DATE'] = 'Date'
    elif 'date' in df.columns: column_rename_map['date'] = 'Date'
    # Latitude column
    if 'LATITUDE' in df.columns: column_rename_map['LATITUDE'] = 'Latitude'
    elif 'latitude' in df.columns: column_rename_map['latitude'] = 'Latitude'
    # Longitude column
    if 'LONGITUDE' in df.columns: column_rename_map['LONGITUDE'] = 'Longitude'
    elif 'longitude' in df.columns: column_rename_map['longitude'] = 'Longitude'
    # Value column
    if relevant_col_name in df.columns: column_rename_map[relevant_col_name] = met_var_name

    df.rename(columns=column_rename_map, inplace=True)

    # Check if essential columns are present after renaming
    required_cols = ['Date', 'Latitude', 'Longitude', met_var_name]
    if not all(col in df.columns for col in required_cols):
        print(f"Warning: Missing one or more required columns ({required_cols}) in {file_path} after renaming. Skipping.")
        # print(f"Available columns: {df.columns.tolist()}")
        return None

    # Ensure correct data types and handle errors by coercing to NaN
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
    df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
    df[met_var_name] = pd.to_numeric(df[met_var_name], errors='coerce')

    # Drop rows where essential identifiers or the value itself are NaN after conversion
    df.dropna(subset=['Date', 'Latitude', 'Longitude', met_var_name], inplace=True)

    if df.empty:
        # print(f"Warning: No valid data left in {file_path} after cleaning. Skipping.")
        return None

    if is_hourly:
        # Aggregate to daily. For Wind Direction, simple mean of degrees is a placeholder.
        # A more accurate method for wind direction involves vector averaging.
        df_daily = df.groupby(['Date', 'Latitude', 'Longitude'], as_index=False)[met_var_name].mean()
        return df_daily
    else:
        # If data is already daily, select relevant columns and remove duplicates if any
        df_daily = df[['Date', 'Latitude', 'Longitude', met_var_name]].drop_duplicates(subset=['Date', 'Latitude', 'Longitude'])
        return df_daily

# --- Function to Load and Merge All Pollutant and Meteorological Data ---
def load_and_merge_all_data(main_pollutant_file, yearly_data_directory, train_years_range):
    """
    Loads the main pollutant data and merges it with yearly meteorological data
    for the specified training years.
    """
    print("--- Starting Data Loading and Merging ---")
    # Load main pollutant data
    try:
        df_pollutants = pd.read_csv(main_pollutant_file)
    except FileNotFoundError:
        print(f"Error: Main pollutant file {main_pollutant_file} not found.")
        return None

    df_pollutants['Date'] = pd.to_datetime(df_pollutants['Date'])

    # Select relevant pollutant columns from the main file
    pollutant_cols = ['Date', 'Latitude', 'Longitude',
                      'Daily Mean PM2.5 Concentration',
                      'Daily Max 1-hour NO2 Concentration', # Using Max as per file
                      'Daily Max 1-hour SO2 Concentration'] # Using Max as per file

    # Check if all pollutant columns exist
    missing_poll_cols = [col for col in pollutant_cols if col not in df_pollutants.columns]
    if missing_poll_cols:
        print(f"Error: Missing required pollutant columns in {main_pollutant_file}: {missing_poll_cols}")
        return None
    df_pollutants = df_pollutants[pollutant_cols]

    # Filter pollutant data for the specified training years
    df_merged = df_pollutants[df_pollutants['Date'].dt.year.isin(train_years_range)].copy()
    if df_merged.empty:
        print(f"Warning: No pollutant data found for the years {train_years_range} in {main_pollutant_file}.")
        # If no base data, we can't proceed with merging
        return pd.DataFrame() # Return an empty DataFrame

    print(f"Loaded pollutant data for {len(df_merged)} records within {train_years_range}.")

    # Configuration for meteorological variables
    met_vars_config = {
        # New Name      File Suffix            Original Column Name in CSV         Is Hourly?
        'Temperature': {'suffix': 'Temperature.csv', 'col': 'DailyAverageDryBulbTemperature', 'hourly': False},
        'WindDir':     {'suffix': 'WindDir.csv',     'col': 'HourlyWindDirection',            'hourly': True},
        'WindSpeed':   {'suffix': 'WindSpeed.csv',   'col': 'HourlyWindSpeed',                'hourly': True},
    }

    # Round Latitude and Longitude in the base merged df for robust merging
    df_merged['Latitude'] = df_merged['Latitude'].round(4)
    df_merged['Longitude'] = df_merged['Longitude'].round(4)

    for met_var_new_name, config in met_vars_config.items():
        all_years_single_met_df_list = []
        print(f"\nProcessing meteorological variable: {met_var_new_name}")
        for year in train_years_range:
            # Construct file path assuming files are named like "YYYY-Suffix.csv"
            file_name = f"{year}-{config['suffix']}"
            file_path = os.path.join(yearly_data_directory, file_name)

            # print(f"Attempting to load: {file_path}")
            df_year_met = load_single_met_file(file_path, met_var_new_name, config['col'], config['hourly'])
            if df_year_met is not None and not df_year_met.empty:
                all_years_single_met_df_list.append(df_year_met)
            # else:
                # print(f"No data loaded for {met_var_new_name} for year {year}.")

        if all_years_single_met_df_list:
            df_all_years_single_met = pd.concat(all_years_single_met_df_list, ignore_index=True)
            print(f"Successfully concatenated {met_var_new_name} data for all years ({len(df_all_years_single_met)} records).")

            # Round lat/lon for merging
            df_all_years_single_met['Latitude'] = df_all_years_single_met['Latitude'].round(4)
            df_all_years_single_met['Longitude'] = df_all_years_single_met['Longitude'].round(4)

            # Perform the merge
            df_merged = pd.merge(df_merged, df_all_years_single_met,
                                 on=['Date', 'Latitude', 'Longitude'],
                                 how='left') # Left join to keep all pollutant records
            print(f"Merged {met_var_new_name}. df_merged now has {len(df_merged)} records.")
        else:
            print(f"Warning: No data successfully loaded for {met_var_new_name} across any specified years. Column will be NaNs.")
            df_merged[met_var_new_name] = np.nan # Add an empty column

    print("--- Data Loading and Merging Complete ---")
    return df_merged


# --- Main Preprocessing Function to Create Model Inputs ---
def preprocess_and_create_model_inputs(
    merged_df,
    target_pollutant_col,
    feature_cols_to_use, # Initial list of features from merged_df to consider
    completeness_threshold=0.7, # Minimum % of data a station must have
    n_neighbors=8, # For GCN graph
    lookback=7     # Timesteps to look back
):
    """
    Processes the merged dataframe: station reliability, pivot, zero-variance removal,
    NaN filling, scaling, and sequence creation.
    """
    print("\n--- Starting Data Preprocessing and Input Creation ---")
    if merged_df is None or merged_df.empty:
        print("Error: Input DataFrame is None or empty. Cannot preprocess.")
        return None, None, None, None, None, None, None, None

    df = merged_df.copy()

    # Ensure feature_cols_to_use actually exist in df
    actual_features_present = [col for col in feature_cols_to_use if col in df.columns]
    if len(actual_features_present) != len(feature_cols_to_use):
        missing_features = [col for col in feature_cols_to_use if col not in df.columns]
        print(f"Warning: Some specified features are not in the merged DataFrame: {missing_features}")
        print(f"Using available features: {actual_features_present}")
    feature_cols_to_use = actual_features_present
    if not feature_cols_to_use or target_pollutant_col not in feature_cols_to_use:
        print(f"Error: Target pollutant '{target_pollutant_col}' or other essential features are missing from the merged data.")
        return None, None, None, None, None, None, None, None


    # 1. Station Reliability Check
    # Create a unique station identifier string for grouping (handles float precision)
    df['station_uid_str'] = df['Latitude'].round(4).astype(str) + '_' + df['Longitude'].round(4).astype(str)

    unique_dates_in_period = df['Date'].unique()
    total_days_in_period = len(unique_dates_in_period)

    if total_days_in_period == 0:
        print("Error: No unique dates found in the data. Cannot perform reliability check.")
        return None, None, None, None, None, None, None, None

    # Calculate completeness based on the target pollutant
    station_completeness = df.groupby('station_uid_str')[target_pollutant_col].count() / total_days_in_period
    reliable_station_uids = station_completeness[station_completeness >= completeness_threshold].index.tolist()

    if not reliable_station_uids:
        raise ValueError(f"No stations meet the completeness threshold of {completeness_threshold*100}%. Try lowering it or check data merging.")

    print(f"--- Station Reliability Check (Threshold: {completeness_threshold*100}%) ---")
    print(f"Total unique station UIDs found before filtering: {df['station_uid_str'].nunique()}")
    print(f"Keeping {len(reliable_station_uids)} reliable stations.")

    df = df[df['station_uid_str'].isin(reliable_station_uids)].copy()
    if df.empty:
        print("Error: No data left after station reliability filtering.")
        return None, None, None, None, None, None, None, None

    # Create a final integer `location_id` for the reliable stations
    locations_df = df[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
    locations_df['location_id'] = locations_df.index

    # Merge this integer location_id back
    df = pd.merge(df, locations_df, on=['Latitude', 'Longitude'], how='left')
    print(f"Data reduced to {len(df)} records after reliability filtering and location_id mapping.")

    # 2. Pivot Data
    # Pivoting with `location_id` as columns and `Date` as index. Values are the features.
    try:
        data_pivot = df.pivot_table(index='Date', columns='location_id',
                                    values=feature_cols_to_use, aggfunc='mean')
    except Exception as e:
        print(f"Error during pivot: {e}")
        print("Check for duplicate (Date, location_id) entries or issues with feature_cols_to_use.")
        # print(df[df.duplicated(subset=['Date', 'location_id'], keep=False)])
        return None, None, None, None, None, None, None, None

    print(f"Pivoted data shape: {data_pivot.shape}")


    # 3. Zero-Variance Feature Removal (applied to pivoted data)
    # Columns in data_pivot are MultiIndex: (feature_name, location_id)
    # We check variance for each base feature_name across all its locations and times.
    current_features_in_pivot = data_pivot.columns.get_level_values(0).unique().tolist()
    final_feature_cols_for_model = list(current_features_in_pivot) # Start with all pivoted features

    print("\n--- Checking for Zero-Variance Features Post-Pivot ---")
    for feature_name in current_features_in_pivot:
        feature_data_matrix = data_pivot[feature_name] # This selects all location columns for that feature
        # Check if the entire block of data for this feature (all its locations over all time) has zero variance
        if feature_data_matrix.to_numpy(na_value=np.nan).var() == 0: # Use nanvar if NaNs are expected
            print(f"🚩 Found zero-variance feature post-pivot: '{feature_name}'. It will be removed.")
            if feature_name in final_feature_cols_for_model:
                final_feature_cols_for_model.remove(feature_name)
            # Drop from pivot table
            data_pivot = data_pivot.drop(columns=feature_name, level=0)

    if target_pollutant_col not in final_feature_cols_for_model:
        raise ValueError(f"Target pollutant '{target_pollutant_col}' was removed due to zero variance or missing. Cannot proceed.")
    print(f"✅ Final features for model: {final_feature_cols_for_model}\n")


    # 4. Fill NaNs (Crucial after pivoting)
    # Sort columns to ensure ffill/bfill are predictable (feature then location_id)
    data_pivot = data_pivot.sort_index(axis=1)

    # Fill across time first (axis=0) for each (feature, location) series
    data_pivot.ffill(axis=0, inplace=True)
    data_pivot.bfill(axis=0, inplace=True)

    # Interpolate for any remaining gaps in time series
    data_pivot.interpolate(method='linear', axis=0, limit_direction='both', inplace=True)

    # If NaNs still exist (e.g., a station has NO data for a feature over the entire period),
    # fill them. Filling with 0 is a last resort. Mean of the feature could be better.
    if data_pivot.isnull().values.any():
        print("Warning: Data still contains NaNs after ffill/bfill/interpolate. Filling remaining with 0.")
        data_pivot.fillna(0, inplace=True)


    # 5. Normalize Data
    # The data_pivot columns are MultiIndex (feature_name, location_id).
    # The scaler will learn a mean/std for each of these individual series.
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(data_pivot.to_numpy())
    df_scaled_pivot = pd.DataFrame(scaled_values, index=data_pivot.index, columns=data_pivot.columns)

    # 6. Reshape Scaled Data to 3D Tensor: (num_dates, num_locations, num_features)
    num_dates = len(df_scaled_pivot.index)
    num_reliable_locations = len(locations_df) # Based on reliable stations
    num_final_model_features = len(final_feature_cols_for_model)

    final_3d_data_np = np.zeros((num_dates, num_reliable_locations, num_final_model_features))

    for i, feature_name_model in enumerate(final_feature_cols_for_model):
        for j, loc_id_model in enumerate(locations_df['location_id']): # loc_id_model is 0, 1, ..., N-1
            # The column in df_scaled_pivot is (feature_name_model, loc_id_model)
            if (feature_name_model, loc_id_model) in df_scaled_pivot.columns:
                final_3d_data_np[:, j, i] = df_scaled_pivot[(feature_name_model, loc_id_model)].values
            else:
                # This might happen if a feature was dropped for a specific location due to all NaNs,
                # or if a feature was entirely dropped (handled by zero-var check).
                # Should be less common with robust NaN filling.
                print(f"Warning: Column ({feature_name_model}, {loc_id_model}) not found in scaled pivot during 3D array construction. Filling with 0.")
                final_3d_data_np[:, j, i] = 0 # Or some other imputation

    # 7. Create Graph Structure and Input/Output Sequences
    adj_matrix = kneighbors_graph(locations_df[['Latitude', 'Longitude']],
                                  n_neighbors=min(n_neighbors, num_reliable_locations -1 if num_reliable_locations >1 else 1), # n_neighbors can't be > n_points-1
                                  mode='connectivity', include_self=True)
    edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)

    X_list, y_list = [], []
    try:
        target_feature_idx_in_model = final_feature_cols_for_model.index(target_pollutant_col)
    except ValueError:
        raise ValueError(f"Target pollutant '{target_pollutant_col}' not in final model features: {final_feature_cols_for_model}")

    for i in range(len(final_3d_data_np) - lookback):
        X_list.append(final_3d_data_np[i : i + lookback, :, :])
        y_list.append(final_3d_data_np[i + lookback, :, target_feature_idx_in_model])

    if not X_list or not y_list:
        print("Error: Not enough data to create lookback sequences. Consider a smaller lookback or more data.")
        return None, None, None, None, None, None, None, None

    X_tensor = torch.tensor(np.array(X_list), dtype=torch.float32)
    y_tensor = torch.tensor(np.array(y_list), dtype=torch.float32)

    print("--- Preprocessing and Input Creation Complete ---")
    return (X_tensor, y_tensor, edge_index, scaler, locations_df,
            num_reliable_locations, num_final_model_features, final_feature_cols_for_model)


# --- Main Script Flow (Example) ---
if __name__ == '__main__':
    # Define paths and parameters
    base_data_directory_path = '/content/drive/MyDrive/pcdc'
    main_pollutant_file_path = 'Master99-23.csv' # Your main pollutant data
    # IMPORTANT: Replace with the ACTUAL path to the directory containing yearly CSVs
    # e.g., '/content/data/met_yearly_csvs/' or 'met_data/' if in the same directory
    yearly_met_data_dir = '.' # Placeholder: current directory. Update this!

    train_years = range(2010, 2023) # Train on 2010-2022

    # Load and merge all data sources
    df_merged_all_years = load_and_merge_all_data(main_pollutant_file_path,
                                                  yearly_met_data_dir,
                                                  train_years)

    if df_merged_all_years is not None and not df_merged_all_years.empty:
        print(f"\nTotal records in merged DataFrame for 2010-2022: {len(df_merged_all_years)}")
        print(f"Columns in merged DataFrame: {df_merged_all_years.columns.tolist()}")
        # print(df_merged_all_years.head())
        # print(df_merged_all_years.isnull().sum())


        # Define features to use from the merged DataFrame
        # These must match column names AFTER merging and any renaming in load_single_met_file
        initial_features_for_model = [
            'Daily Mean PM2.5 Concentration',
            'Daily Max 1-hour NO2 Concentration',
            'Daily Max 1-hour SO2 Concentration',
            'Temperature',
            'WindSpeed',
            'WindDir'
        ]
        target_col = 'Daily Mean PM2.5 Concentration'

        # Preprocess the merged data to create model inputs
        processed_outputs = preprocess_and_create_model_inputs(
            merged_df=df_merged_all_years,
            target_pollutant_col=target_col,
            feature_cols_to_use=initial_features_for_model,
            completeness_threshold=0.6, # Adjusted threshold, can be tuned
            n_neighbors=4,              # Can be tuned
            lookback=7                  # Standard lookback period
        )

        if processed_outputs[0] is not None: # Check if X_tensor is not None
            X, y, edge_idx, data_scaler, reliable_locs_df, \
            n_locs, n_feats, final_feats_list = processed_outputs

            print("\n--- Final Processed Data Shapes ---")
            print(f"X shape: {X.shape}")
            print(f"y shape: {y.shape}")
            print(f"edge_index shape: {edge_idx.shape}")
            print(f"Number of reliable locations: {n_locs}")
            print(f"Number of features in model: {n_feats}")
            print(f"Final features list: {final_feats_list}")
            print(f"Reliable stations info:\n{reliable_locs_df.head()}")

            # Now you can proceed to split X and y into train/val/test sets,
            # create DataLoaders, define the PCDCNet model (adjusting input_dim if n_feats changed),
            # and train the model.
            # Example:
            # train_size = int(0.8 * len(X))
            # X_train, y_train = X[:train_size], y[:train_size]
            # ... and so on for validation/test split

            # Remember to pass n_locs and n_feats to your PCDCNet model instantiation.
            # model = PCDCNet(num_features=n_feats, num_locations=n_locs)

        else:
            print("Failed to create model inputs due to errors in preprocessing.")
    else:
        print("Failed to load and merge data. Cannot proceed with preprocessing.")

## PCDCNet Model Architecture

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np

# --- 1. PCDCNet Model Definition ---
# This is the model architecture you provided. It is a solid implementation
# for this task.
class PCDCNet(nn.Module):
    def __init__(self, num_features, num_locations, hidden_dim=64, gcn_out_dim=32):
        super(PCDCNet, self).__init__()

        # MLP for local feature enhancement
        self.mlp = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # Graph Convolutional Network for spatial dependencies
        self.gcn = GCNConv(hidden_dim, gcn_out_dim)

        # GRU for temporal dependencies
        self.gru = nn.GRU(gcn_out_dim, hidden_dim, batch_first=True)

        # Output layer
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, edge_index):
        # x shape: (batch_size, lookback, num_locations, num_features)
        batch_size, lookback, num_loc, num_feat = x.shape

        # Process each time step
        gru_inputs = []
        for t in range(lookback):
            # Current time step data for all locations in the batch
            x_t = x[:, t, :, :]
            x_t = x_t.reshape(batch_size * num_loc, num_feat)

            # Local feature enhancement with MLP
            mlp_out = self.mlp(x_t)

            # Reshape for GCN
            mlp_out = mlp_out.reshape(batch_size, num_loc, -1)

            # GCN for spatial modeling
            # We process each graph in the batch individually
            gcn_batch_out = []
            for i in range(batch_size):
                gcn_out = self.gcn(mlp_out[i], edge_index)
                gcn_batch_out.append(gcn_out)

            gcn_out_tensor = torch.stack(gcn_batch_out)
            gru_inputs.append(gcn_out_tensor)

        # Stack to create a sequence for the GRU
        # Shape: (batch_size, lookback, num_loc, gcn_out_dim)
        gru_input_seq = torch.stack(gru_inputs, dim=1)

        # Reshape to combine batch and location dimensions for GRU processing
        # The GRU will treat each location's time series independently
        gru_input_seq = gru_input_seq.reshape(batch_size * num_loc, lookback, -1)

        # GRU for temporal modeling
        gru_out, _ = self.gru(gru_input_seq)

        # Use the output of the last time step
        last_hidden_state = gru_out[:, -1, :]

        # Reshape back to (batch_size, num_loc, hidden_dim)
        last_hidden_state = last_hidden_state.reshape(batch_size, num_loc, -1)

        # Fully connected layer for the final prediction
        output = self.fc(last_hidden_state)

        return output.squeeze(-1)


# --- 2. Model Training Function ---
def train_model(model, train_loader, val_loader, edge_index, device, epochs=50, learning_rate=0.001):
    """
    Trains the PCDCNet model with gradient clipping for stability.
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    val_losses = []

    print("\n--- Starting Model Training ---")
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs, edge_index)
            loss = criterion(outputs, labels)

            if torch.isnan(loss):
                print(f"NaN detected in loss at epoch {epoch+1}. Stopping training.")
                return train_losses, val_losses

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Gradient clipping
            optimizer.step()

            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # Validation loop
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs, edge_index)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

        val_loss = val_running_loss / len(val_loader)
        val_losses.append(val_loss)

        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')

    print("--- Model Training Complete ---")
    return train_losses, val_losses


# --- 3. Model Evaluation Function ---
def evaluate_model(model, test_loader, edge_index, device, scaler_obj, num_locations, num_features, target_feature_idx):
    """
    Evaluates the model on the test set and returns metrics on the original data scale.
    """
    model.eval()
    all_preds = []
    all_labels = []
    print("\n--- Evaluating Model on Test Set ---")
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs, edge_index)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    # Concatenate all batches
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # To inverse transform, we need to reconstruct a dummy array with the same shape
    # as the one the scaler was fit on: (num_samples, num_features * num_locations)
    # Flatten the predictions and labels
    all_preds_flat = all_preds.flatten()
    all_labels_flat = all_labels.flatten()

    # Create dummy arrays for inverse scaling
    dummy_preds_array = np.zeros((len(all_preds_flat), num_features * num_locations))
    dummy_labels_array = np.zeros((len(all_labels_flat), num_features * num_locations))

    # Place the predictions/labels into the correct column for the target feature
    # The scaler expects data in the order of (feature1_loc1, feature1_loc2, ..., feature2_loc1, ...)
    # The target feature is PM2.5, which is at index `target_feature_idx` in our feature list
    num_total_series = num_features * num_locations
    target_cols_indices = [i for i in range(num_total_series) if (i % num_features) == target_feature_idx]

    # This is a bit complex, but necessary for the scaler. It places the flattened predictions
    # back into the columns that correspond to the PM2.5 data for each location.
    dummy_preds_array = np.zeros((all_preds.shape[0], num_total_series))
    dummy_labels_array = np.zeros((all_labels.shape[0], num_total_series))

    # Reshape the predictions and labels to match the number of samples and locations
    all_preds_reshaped = all_preds.reshape(-1, num_locations)
    all_labels_reshaped = all_labels.reshape(-1, num_locations)

    # Assign the reshaped data to the appropriate columns in the dummy arrays
    for i in range(num_locations):
         dummy_preds_array[:, target_feature_idx + i * num_features] = all_preds_reshaped[:, i]
         dummy_labels_array[:, target_feature_idx + i * num_features] = all_labels_reshaped[:, i]

    # Inverse transform
    preds_rescaled_full = scaler_obj.inverse_transform(dummy_preds_array)
    labels_rescaled_full = scaler_obj.inverse_transform(dummy_labels_array)

    # Extract only the columns corresponding to the target feature
    preds_rescaled = preds_rescaled_full[:, target_feature_idx::num_features].flatten()
    labels_rescaled = labels_rescaled_full[:, target_feature_idx::num_features].flatten()

    # Calculate metrics
    mae = np.mean(np.abs(preds_rescaled - labels_rescaled))
    rmse = np.sqrt(np.mean((preds_rescaled - labels_rescaled)**2))

    # --- ADDED ACCURACY METRIC (R-squared) ---
    # It measures how well the model predictions explain the variance of the true values.
    # A score of 1.0 is a perfect prediction.
    ss_res = np.sum((labels_rescaled - preds_rescaled) ** 2)
    ss_tot = np.sum((labels_rescaled - np.mean(labels_rescaled)) ** 2)
    # Add a check to prevent division by zero if all target values are the same
    if ss_tot == 0:
        r2 = 0.0 # Or handle as an undefined case
    else:
        r2 = 1 - (ss_res / ss_tot)

    return mae, rmse, r2


# --- 4. Main Execution Block ---
# This block assumes the variables from the previous preprocessing script are available
# (X, y, edge_idx, data_scaler, n_locs, n_feats, final_feats_list)

try:
    # Check if the required variables from preprocessing exist
    X, y, edge_idx, data_scaler, n_locs, n_feats, final_feats_list
except NameError:
    print("Error: Preprocessing variables (X, y, etc.) not found.")
    print("Please run the data processing script cell before this one.")
else:
    # Setup device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Split data: 80% train, 10% validation, 10% test
    train_size = int(0.8 * len(X))
    val_size = int(0.1 * len(X))
    test_size = len(X) - train_size - val_size

    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
    X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

    # Create DataLoaders
    batch_size = 32
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Instantiate the model using the variables from your preprocessing
    model = PCDCNet(num_features=n_feats, num_locations=n_locs).to(device)
    edge_idx = edge_idx.to(device)
    print("\n--- Model Architecture ---")
    print(model)
    print("--------------------------\n")

    # Train the model
    train_losses, val_losses = train_model(model, train_loader, val_loader, edge_idx, device, epochs=50)

    # Plot training and validation loss
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training & Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('MSE Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Evaluate the final model on the test set
    target_feature_idx = final_feats_list.index('Daily Mean PM2.5 Concentration')
    # --- UPDATED TO RECEIVE r2 ---
    mae, rmse, r2 = evaluate_model(model, test_loader, edge_idx, device, data_scaler, n_locs, n_feats, target_feature_idx)

    # --- UPDATED TO PRINT r2 ---
    print(f"\n--- Final Evaluation Metrics ---")
    print(f"Test MAE (Mean Absolute Error): {mae:.4f}")
    print(f"Test RMSE (Root Mean Squared Error): {rmse:.4f}")
    print(f"Test R-squared (Accuracy): {r2:.4f}")
    print("--------------------------------")


# PCDC Using LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np

# --- 1. PCDCNet Model Definition (Now with LSTM) ---
class PCDCNet(nn.Module):
    def __init__(self, num_features, num_locations, hidden_dim=64, gcn_out_dim=32):
        super(PCDCNet, self).__init__()

        # MLP for local feature enhancement
        self.mlp = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # Graph Convolutional Network for spatial dependencies
        self.gcn = GCNConv(hidden_dim, gcn_out_dim)

        # --- MODIFICATION: Using LSTM instead of GRU ---
        # The LSTM layer has the same parameters for this use case.
        self.lstm = nn.LSTM(gcn_out_dim, hidden_dim, batch_first=True)
        # --- END OF MODIFICATION ---

        # Output layer
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, edge_index):
        # x shape: (batch_size, lookback, num_locations, num_features)
        batch_size, lookback, num_loc, num_feat = x.shape

        # Process each time step
        lstm_inputs = []
        for t in range(lookback):
            # Current time step data for all locations in the batch
            x_t = x[:, t, :, :]
            x_t = x_t.reshape(batch_size * num_loc, num_feat)

            # Local feature enhancement with MLP
            mlp_out = self.mlp(x_t)

            # Reshape for GCN
            mlp_out = mlp_out.reshape(batch_size, num_loc, -1)

            # GCN for spatial modeling
            gcn_batch_out = []
            for i in range(batch_size):
                gcn_out = self.gcn(mlp_out[i], edge_index)
                gcn_batch_out.append(gcn_out)

            gcn_out_tensor = torch.stack(gcn_batch_out)
            lstm_inputs.append(gcn_out_tensor)

        # Stack to create a sequence for the LSTM
        lstm_input_seq = torch.stack(lstm_inputs, dim=1)
        lstm_input_seq = lstm_input_seq.reshape(batch_size * num_loc, lookback, -1)

        # --- MODIFICATION: Calling the LSTM layer ---
        # The LSTM returns the output and a tuple of (hidden_state, cell_state).
        # We only need the main output sequence for this logic.
        lstm_out, _ = self.lstm(lstm_input_seq)
        # --- END OF MODIFICATION ---

        # Use the output of the last time step
        last_hidden_state = lstm_out[:, -1, :]

        # Reshape back to (batch_size, num_loc, hidden_dim)
        last_hidden_state = last_hidden_state.reshape(batch_size, num_loc, -1)

        # Fully connected layer for the final prediction
        output = self.fc(last_hidden_state)

        return output.squeeze(-1)


# --- 2. Model Training Function (No Changes Needed) ---
def train_model(model, train_loader, val_loader, edge_index, device, epochs=50, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_losses, val_losses = [], []

    print("\n--- Starting Model Training ---")
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, edge_index)
            loss = criterion(outputs, labels)
            if torch.isnan(loss):
                print(f"NaN detected in loss at epoch {epoch+1}. Stopping training.")
                return train_losses, val_losses
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item()
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs, edge_index)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
        val_loss = val_running_loss / len(val_loader)
        val_losses.append(val_loss)

        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')

    print("--- Model Training Complete ---")
    return train_losses, val_losses


# --- 3. Model Evaluation Function (No Changes Needed) ---
def evaluate_model(model, test_loader, edge_index, device, scaler_obj, num_locations, num_features, target_feature_idx):
    model.eval()
    all_preds, all_labels = [], []
    print("\n--- Evaluating Model on Test Set ---")
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs, edge_index)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    num_total_original_features = scaler_obj.n_features_in_
    dummy_preds_array = np.zeros((all_preds.shape[0], num_total_original_features))
    dummy_labels_array = np.zeros((all_labels.shape[0], num_total_original_features))
    start_col_idx = target_feature_idx * num_locations
    end_col_idx = start_col_idx + num_locations
    dummy_preds_array[:, start_col_idx:end_col_idx] = all_preds
    dummy_labels_array[:, start_col_idx:end_col_idx] = all_labels
    preds_rescaled_full = scaler_obj.inverse_transform(dummy_preds_array)
    labels_rescaled_full = scaler_obj.inverse_transform(dummy_labels_array)
    preds_rescaled = preds_rescaled_full[:, start_col_idx:end_col_idx].flatten()
    labels_rescaled = labels_rescaled_full[:, start_col_idx:end_col_idx].flatten()
    mae = np.mean(np.abs(preds_rescaled - labels_rescaled))
    rmse = np.sqrt(np.mean((preds_rescaled - labels_rescaled)**2))
    ss_res = np.sum((labels_rescaled - preds_rescaled)**2)
    ss_tot = np.sum((labels_rescaled - np.mean(labels_rescaled))**2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0.0
    return mae, rmse, r2


# --- 4. Main Execution Block (No Changes Needed) ---
# This block assumes the variables from your preprocessing script are available
try:
    X, y, edge_idx, data_scaler, n_locs, n_feats, final_feats_list
except NameError:
    print("Error: Preprocessing variables (X, y, etc.) not found.")
    print("Please run the data processing script cell before this one.")
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Data splitting
    train_size = int(0.8 * len(X))
    val_size = int(0.1 * len(X))
    test_size = len(X) - train_size - val_size
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
    X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

    # DataLoaders
    batch_size = 32
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Instantiate the model with the LSTM layer
    model = PCDCNet(num_features=n_feats, num_locations=n_locs).to(device)
    edge_idx = edge_idx.to(device)
    print("\n--- Model Architecture (LSTM Version) ---")
    print(model)
    print("---------------------------------------\n")

    # Train the model
    train_losses, val_losses = train_model(model, train_loader, val_loader, edge_idx, device, epochs=50)

    # Plot loss
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training & Validation Loss (LSTM Model)')
    plt.xlabel('Epochs')
    plt.ylabel('MSE Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Evaluate the final model
    target_feature_idx = final_feats_list.index('Daily Mean PM2.5 Concentration')
    mae, rmse, r2 = evaluate_model(model, test_loader, edge_idx, device, data_scaler, n_locs, n_feats, target_feature_idx)

    print(f"\n--- Final Evaluation Metrics (LSTM Model) ---")
    print(f"Test MAE (Mean Absolute Error): {mae:.4f}")
    print(f"Test RMSE (Root Mean Squared Error): {rmse:.4f}")
    print(f"Test R-squared (Accuracy): {r2:.4f}")
    print("-------------------------------------------")


In [ ]:
import torch
import joblib
import os

# --- This assumes these variables exist in your current Colab session after training ---
# model: your trained PCDCNet model
# X, y, edge_idx: Tensors from the preprocessing step
# data_scaler: The StandardScaler object from preprocessing
# n_locs, n_feats, final_feats_list: Variables from preprocessing

# Define the paths to save the files in your 'pcdc' folder
save_directory = '/content/drive/MyDrive/pcdc/'
model_path = os.path.join(save_directory, 'pcdcnet_model.pth')
data_path = os.path.join(save_directory, 'evaluation_data.pt')
scaler_path = os.path.join(save_directory, 'scaler.gz')

print("--- Saving model and CORRECT evaluation data ---")

# 1. Save the trained model's state dictionary
torch.save(model.state_dict(), model_path)
print(f"Model saved to: {model_path}")

# 2. Save the data scaler
joblib.dump(data_scaler, scaler_path)
print(f"Scaler saved to: {scaler_path}")

# 3. Save all other necessary tensors and variables
evaluation_data = {
    'X': X,
    'y': y,
    'edge_idx': edge_idx,
    'n_locs': n_locs,
    'n_feats': n_feats,
    'final_feats_list': final_feats_list
}
torch.save(evaluation_data, data_path)
print(f"Evaluation data saved to: {data_path}")

print("--- All files have been re-saved and are now synchronized! ---")

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import joblib
import os

# --- PCDCNet Model Definition (Using GRU to match your saved model) ---
class PCDCNet(nn.Module):
    def __init__(self, num_features, num_locations, hidden_dim=64, gcn_out_dim=32):
        super(PCDCNet, self).__init__()
        self.mlp = nn.Sequential(nn.Linear(num_features, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, hidden_dim))
        self.gcn = GCNConv(hidden_dim, gcn_out_dim)
        # --- Using GRU to match the architecture of the saved model weights ---
        self.gru = nn.GRU(gcn_out_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, edge_index):
        batch_size, lookback, num_loc, num_feat = x.shape
        recurrent_inputs = []
        for t in range(lookback):
            x_t = x[:, t, :, :].reshape(batch_size * num_loc, num_feat)
            mlp_out = self.mlp(x_t).reshape(batch_size, num_loc, -1)
            gcn_batch_out = []
            for i in range(batch_size):
                gcn_batch_out.append(self.gcn(mlp_out[i], edge_index))
            recurrent_inputs.append(torch.stack(gcn_batch_out))

        recurrent_input_seq = torch.stack(recurrent_inputs, dim=1).reshape(batch_size * num_loc, lookback, -1)
        # --- Calling the GRU layer ---
        recurrent_out, _ = self.gru(recurrent_input_seq)
        last_hidden_state = recurrent_out[:, -1, :].reshape(batch_size, num_loc, -1)
        output = self.fc(last_hidden_state)
        return output.squeeze(-1)

# --- CORRECTED Model Evaluation Function ---
def evaluate_model(model, test_loader, edge_index, device, scaler_obj, num_locations, num_features, target_feature_idx):
    model.eval()
    all_preds, all_labels = [], []
    print("\n--- Evaluating Model on Test Set ---")
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs, edge_index)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # --- ADDED ROBUSTNESS CHECK ---
    # Infer the number of locations directly from the model's output shape
    n_locs_from_model = all_preds.shape[1]

    # Check for a mismatch between the model's output and the loaded data parameters
    if n_locs_from_model != num_locations:
        print("\n" + "="*60)
        print("!! FATAL ERROR: Data and Model Mismatch !!")
        print(f"The loaded model is making predictions for {n_locs_from_model} locations.")
        print(f"However, the loaded data/scaler object is for {num_locations} locations.")
        print("This means the saved model file and the saved data/scaler files are out of sync.")
        print("\nSOLUTION: Please re-run the 'Save Model & Data' script in the")
        print("Colab notebook session where you successfully trained the model.")
        print("="*60 + "\n")
        return None, None, None # Stop evaluation
    # --- END OF CHECK ---

    num_total_original_features = scaler_obj.n_features_in_
    dummy_preds_array = np.zeros((all_preds.shape[0], num_total_original_features))
    dummy_labels_array = np.zeros((all_labels.shape[0], num_total_original_features))

    start_col_idx = target_feature_idx * num_locations
    end_col_idx = start_col_idx + num_locations

    dummy_preds_array[:, start_col_idx:end_col_idx] = all_preds
    dummy_labels_array[:, start_col_idx:end_col_idx] = all_labels

    preds_rescaled_full = scaler_obj.inverse_transform(dummy_preds_array)
    labels_rescaled_full = scaler_obj.inverse_transform(dummy_labels_array)

    preds_rescaled = preds_rescaled_full[:, start_col_idx:end_col_idx].flatten()
    labels_rescaled = labels_rescaled_full[:, start_col_idx:end_col_idx].flatten()

    mae = np.mean(np.abs(preds_rescaled - labels_rescaled))
    rmse = np.sqrt(np.mean((preds_rescaled - labels_rescaled)**2))
    ss_res = np.sum((labels_rescaled - preds_rescaled)**2)
    ss_tot = np.sum((labels_rescaled - np.mean(labels_rescaled))**2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0.0
    return mae, rmse, r2

# --- Main Evaluation Script ---
# Define paths to the saved files in your Google Drive
load_directory = '/content/drive/MyDrive/pcdc/'
model_path = os.path.join(load_directory, 'pcdcnet_model.pth')
data_path = os.path.join(load_directory, 'evaluation_data.pt')
scaler_path = os.path.join(load_directory, 'scaler.gz')

# Check if all files exist before proceeding
if not all(os.path.exists(p) for p in [model_path, data_path, scaler_path]):
    print("Error: One or more required files are missing.")
    print("Please ensure you have run a script to save the trained model, scaler, and data tensors.")
else:
    # Setup device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 1. Load the data objects
    eval_data = torch.load(data_path)
    X = eval_data['X']
    y = eval_data['y']
    edge_idx = eval_data['edge_idx'].to(device)
    n_locs = eval_data['n_locs']
    n_feats = eval_data['n_feats']
    final_feats_list = eval_data['final_feats_list']
    print("Loaded evaluation data and tensors.")

    # 2. Load the data scaler
    data_scaler = joblib.load(scaler_path)
    print("Loaded data scaler.")

    # 3. Instantiate the GRU model and load the saved weights
    model = PCDCNet(num_features=n_feats, num_locations=n_locs).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    print("Instantiated GRU model and loaded trained weights.")

    # 4. Recreate the test data split and DataLoader
    train_size = int(0.8 * len(X))
    val_size = int(0.1 * len(X))
    X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    print("Recreated test data loader.")

    # 5. Run the evaluation
    target_feature_idx = final_feats_list.index('Daily Mean PM2.5 Concentration')
    mae, rmse, r2 = evaluate_model(model, test_loader, edge_idx, device, data_scaler, n_locs, n_feats, target_feature_idx)

    # 6. Print the final metrics
    if mae is not None: # Check if evaluation was successful
        print(f"\n--- Final Evaluation Metrics ---")
        print(f"Test MAE (Mean Absolute Error): {mae:.4f}")
        print(f"Test RMSE (Root Mean Squared Error): {rmse:.4f}")
        print(f"Test R-squared (Accuracy): {r2:.4f}")
        print("--------------------------------")
